# Configuring Deeplay Objects

The `.configure()` method permits you to configure Deeplay objects. You can combine it with selectors to target specific objects.

In [1]:
import deeplay as dl
import torch

## Using the `.configure()` Method

The `.configure()` method exists for all `DeeplayModule` objects. It works by changing the input parameters of the constructor of the class, and subsequently re-initializing the class. 

In [2]:
mlp = dl.MultiLayerPerceptron(
    in_features=10,
    hidden_features=[20, 30, 40],
    out_features=5,
)

print(mlp)

MultiLayerPerceptron(
  (blocks): LayerList(
    (0): LinearBlock(
      (layer): Layer[Linear](in_features=10, out_features=20, bias=True)
      (activation): Layer[ReLU]()
    )
    (1): LinearBlock(
      (layer): Layer[Linear](in_features=20, out_features=30, bias=True)
      (activation): Layer[ReLU]()
    )
    (2): LinearBlock(
      (layer): Layer[Linear](in_features=30, out_features=40, bias=True)
      (activation): Layer[ReLU]()
    )
    (3): LinearBlock(
      (layer): Layer[Linear](in_features=40, out_features=5, bias=True)
      (activation): Layer[Identity]()
    )
  )
)


In [3]:
mlp.configure(out_activation=torch.nn.Tanh)

print(mlp)

MultiLayerPerceptron(
  (blocks): LayerList(
    (0): LinearBlock(
      (layer): Layer[Linear](in_features=10, out_features=20, bias=True)
      (activation): Layer[ReLU]()
    )
    (1): LinearBlock(
      (layer): Layer[Linear](in_features=20, out_features=30, bias=True)
      (activation): Layer[ReLU]()
    )
    (2): LinearBlock(
      (layer): Layer[Linear](in_features=30, out_features=40, bias=True)
      (activation): Layer[ReLU]()
    )
    (3): LinearBlock(
      (layer): Layer[Linear](in_features=40, out_features=5, bias=True)
      (activation): Layer[Tanh]()
    )
  )
)


For `Layer` classes, the configure method will instead affect the constructor argument of the classtype of the layer. For example, `Layer(nn.Conv2d, 8, 8).configure(kernel_size=5)`. The first positional argument can be used to change the class type of the layer.

In [4]:
layer = dl.Layer(
    torch.nn.Conv2d, 
    in_channels=3, 
    out_channels=10, 
    kernel_size=1,
)

print(layer)

Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1)


In [5]:
layer.configure(stride=2)

print(layer)

Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=2)


In [6]:
layer = dl.Layer(torch.nn.ReLU)

print(layer)

Layer[ReLU]()


In [7]:
layer.configure(torch.nn.LeakyReLU, negative_slope=0.1)

print(layer)

Layer[LeakyReLU](negative_slope=0.1)


## Using Selections

Selections are a way to apply an operation or configuration to multiple classes at once. There is some special syntax, which is easier to understand through examples ...

In [8]:
model = dl.UNet2d(
    in_channels=3,
    encoder_channels=[16, 32],
    out_channels=10,
    out_activation=torch.nn.Tanh,
)

print(model)

UNet2d(
  (encoder): ConvolutionalEncoder2d(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (layer): Layer[Conv2d](in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        (activation): Layer[ReLU]()
      )
      (1): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[ReLU]()
      )
    )
    (postprocess): Layer[Identity]()
  )
  (bottleneck): ConvolutionalNeuralNetwork(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[ReLU]()
        (upsample): Layer[ConvTranspose2d](kernel_size=2, stride=2, padding=0, in_channels=32, out_channels=32)
      )
    )
  )
  (decoder): ConvolutionalDecoder2d(
    (blocks): LayerList(
      (0): 

... use strings to select a direct child ...

In [9]:
selection = model["encoder"]

print(selection.list_names())

[('encoder',)]


... use multiple strings to select multiple successive children ...

In [10]:
selection = model["encoder", "blocks"]

print(selection.list_names())

[('encoder', 'blocks')]


... use the `|` operator to select multiple children at the same level ...

In [11]:
selection = model["encoder|decoder"]

print(selection.list_names())

[('encoder',), ('decoder',)]


... use the `:` operator to select all children at the same level ...

In [12]:
selection = model[:]

print(selection.list_names())

[('encoder',), ('bottleneck',), ('decoder',), ('skip',)]


... use the `...` operator to select select all children at any level ...

In [13]:
# This selects all children of the model.
selection = model[...]

print(selection.list_names())

[(), ('encoder',), ('encoder', 'blocks'), ('encoder', 'blocks', '0'), ('encoder', 'blocks', '0', 'layer'), ('encoder', 'blocks', '0', 'activation'), ('encoder', 'blocks', '1'), ('encoder', 'blocks', '1', 'pool'), ('encoder', 'blocks', '1', 'layer'), ('encoder', 'blocks', '1', 'activation'), ('encoder', 'postprocess'), ('bottleneck',), ('bottleneck', 'blocks'), ('bottleneck', 'blocks', '0'), ('bottleneck', 'blocks', '0', 'pool'), ('bottleneck', 'blocks', '0', 'layer'), ('bottleneck', 'blocks', '0', 'activation'), ('bottleneck', 'blocks', '0', 'upsample'), ('decoder',), ('decoder', 'blocks'), ('decoder', 'blocks', '0'), ('decoder', 'blocks', '0', 'layer'), ('decoder', 'blocks', '0', 'activation'), ('decoder', 'blocks', '0', 'upsample'), ('decoder', 'blocks', '1'), ('decoder', 'blocks', '1', 'layer'), ('decoder', 'blocks', '1', 'activation'), ('decoder', 'preprocess'), ('skip',)]


In [14]:
# This selects only the children named "layer".
selection = model[..., "layer"]

print(selection.list_names())

[('encoder', 'blocks', '0', 'layer'), ('encoder', 'blocks', '1', 'layer'), ('bottleneck', 'blocks', '0', 'layer'), ('decoder', 'blocks', '0', 'layer'), ('decoder', 'blocks', '1', 'layer')]


In [15]:
# This selects all children of the "encoder" child.
selection = model["encoder", ...] 

print(selection.list_names())

[('encoder',), ('encoder', 'blocks'), ('encoder', 'blocks', '0'), ('encoder', 'blocks', '0', 'layer'), ('encoder', 'blocks', '0', 'activation'), ('encoder', 'blocks', '1'), ('encoder', 'blocks', '1', 'pool'), ('encoder', 'blocks', '1', 'layer'), ('encoder', 'blocks', '1', 'activation'), ('encoder', 'postprocess')]


In [16]:
# This selects all children at any level, 
# then it selects the children named "blocks", 
# finally it selects all children of the "blocks" children.
selection = model[..., "blocks", ...]

print(selection.list_names())


[('encoder', 'blocks'), ('encoder', 'blocks', '0'), ('encoder', 'blocks', '0', 'layer'), ('encoder', 'blocks', '0', 'activation'), ('encoder', 'blocks', '1'), ('encoder', 'blocks', '1', 'pool'), ('encoder', 'blocks', '1', 'layer'), ('encoder', 'blocks', '1', 'activation'), ('bottleneck', 'blocks'), ('bottleneck', 'blocks', '0'), ('bottleneck', 'blocks', '0', 'pool'), ('bottleneck', 'blocks', '0', 'layer'), ('bottleneck', 'blocks', '0', 'activation'), ('bottleneck', 'blocks', '0', 'upsample'), ('decoder', 'blocks'), ('decoder', 'blocks', '0'), ('decoder', 'blocks', '0', 'layer'), ('decoder', 'blocks', '0', 'activation'), ('decoder', 'blocks', '0', 'upsample'), ('decoder', 'blocks', '1'), ('decoder', 'blocks', '1', 'layer'), ('decoder', 'blocks', '1', 'activation')]


... use the `#` operator to select a subset of previously selected children ...

In [17]:
selection = model[..., "layer#:3"]

print(selection.list_names())

[('encoder', 'blocks', '0', 'layer'), ('encoder', 'blocks', '1', 'layer'), ('bottleneck', 'blocks', '0', 'layer')]


In [18]:
selection = model[..., "layer#1:4"]

print(selection.list_names())

[('encoder', 'blocks', '1', 'layer'), ('bottleneck', 'blocks', '0', 'layer'), ('decoder', 'blocks', '0', 'layer')]


In [19]:
selection = model[..., "layer#-1"]

print(selection.list_names())

[('decoder', 'blocks', '1', 'layer')]


... use the `,` operator to select either of multiple selectors ...

In [20]:
selection = model[..., "layer#0, layer#-1"]

print(selection.list_names())

[('encoder', 'blocks', '0', 'layer'), ('decoder', 'blocks', '1', 'layer')]


... as an example select the first block of the encoder and change its kernel size.

In [21]:
model[..., "layer#0"].configure(kernel_size=1, padding=0)

print(model)

UNet2d(
  (encoder): ConvolutionalEncoder2d(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (layer): Layer[Conv2d](in_channels=3, out_channels=16, kernel_size=1, stride=1, padding=0)
        (activation): Layer[ReLU]()
      )
      (1): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[ReLU]()
      )
    )
    (postprocess): Layer[Identity]()
  )
  (bottleneck): ConvolutionalNeuralNetwork(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[ReLU]()
        (upsample): Layer[ConvTranspose2d](kernel_size=2, stride=2, padding=0, in_channels=32, out_channels=32)
      )
    )
  )
  (decoder): ConvolutionalDecoder2d(
    (blocks): LayerList(
      (0): 

### Filtering a Selection

The selection can be filtered using the `.filter()`, `.hasattr()`, or `.isinstance()` methods.

In [22]:
model = dl.UNet2d(
    in_channels=3,
    encoder_channels=[16, 32],
    out_channels=10,
    out_activation=torch.nn.Tanh,
)

print(model)

UNet2d(
  (encoder): ConvolutionalEncoder2d(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (layer): Layer[Conv2d](in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        (activation): Layer[ReLU]()
      )
      (1): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[ReLU]()
      )
    )
    (postprocess): Layer[Identity]()
  )
  (bottleneck): ConvolutionalNeuralNetwork(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[ReLU]()
        (upsample): Layer[ConvTranspose2d](kernel_size=2, stride=2, padding=0, in_channels=32, out_channels=32)
      )
    )
  )
  (decoder): ConvolutionalDecoder2d(
    (blocks): LayerList(
      (0): 

In [23]:
selection = model[...].isinstance(dl.ConvolutionalEncoder2d)

print(selection.list_names())

[('encoder',)]


In [24]:
selection = model["encoder", ...].isinstance(torch.nn.ReLU)

print(selection.list_names())

[('encoder', 'blocks', '0', 'activation'), ('encoder', 'blocks', '1', 'activation')]


In [25]:
selection = model[...].hasattr("pool")  ### bugged. should not return decoder.

print(selection.list_names())

[('encoder', 'blocks', '0'), ('encoder', 'blocks', '1'), ('bottleneck', 'blocks', '0'), ('decoder', 'blocks', '0'), ('decoder', 'blocks', '1')]


In [26]:
selection = model[...].hasattr("stride")

print(selection.list_names())

[('encoder', 'blocks', '0'), ('encoder', 'blocks', '1'), ('bottleneck', 'blocks', '0'), ('decoder', 'blocks', '0'), ('decoder', 'blocks', '1')]


### Applying Methods to a Selection

Any method can be applied to the selection, not just `.configure()`. This will either apply the method to all classes in the selection, or to the first class in the selection. You can choose which one by accessing the `.all` or `.first` attributes of the selection:

In [27]:
model = dl.UNet2d(
    in_channels=3,
    encoder_channels=[16, 32],
    out_channels=10,
    out_activation=torch.nn.Tanh,
)

print(model)

UNet2d(
  (encoder): ConvolutionalEncoder2d(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (layer): Layer[Conv2d](in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        (activation): Layer[ReLU]()
      )
      (1): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[ReLU]()
      )
    )
    (postprocess): Layer[Identity]()
  )
  (bottleneck): ConvolutionalNeuralNetwork(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[ReLU]()
        (upsample): Layer[ConvTranspose2d](kernel_size=2, stride=2, padding=0, in_channels=32, out_channels=32)
      )
    )
  )
  (decoder): ConvolutionalDecoder2d(
    (blocks): LayerList(
      (0): 

In [28]:
model[..., "layer"].first.configure(kernel_size=5, padding=2)

print(model)

UNet2d(
  (encoder): ConvolutionalEncoder2d(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (layer): Layer[Conv2d](in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        (activation): Layer[ReLU]()
      )
      (1): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[ReLU]()
      )
    )
    (postprocess): Layer[Identity]()
  )
  (bottleneck): ConvolutionalNeuralNetwork(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[ReLU]()
        (upsample): Layer[ConvTranspose2d](kernel_size=2, stride=2, padding=0, in_channels=32, out_channels=32)
      )
    )
  )
  (decoder): ConvolutionalDecoder2d(
    (blocks): LayerList(
      (0): 

In [29]:
model[..., "activation"].all.configure(torch.nn.Sigmoid)

print(model)

UNet2d(
  (encoder): ConvolutionalEncoder2d(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (layer): Layer[Conv2d](in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        (activation): Layer[Sigmoid]()
      )
      (1): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[Sigmoid]()
      )
    )
    (postprocess): Layer[Identity]()
  )
  (bottleneck): ConvolutionalNeuralNetwork(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[Sigmoid]()
        (upsample): Layer[ConvTranspose2d](kernel_size=2, stride=2, padding=0, in_channels=32, out_channels=32)
      )
    )
  )
  (decoder): ConvolutionalDecoder2d(
    (blocks): LayerList(
  

In [30]:
model[...].isinstance(dl.Conv2dBlock).all.normalized(torch.nn.BatchNorm2d)

print(model)


UNet2d(
  (encoder): ConvolutionalEncoder2d(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (layer): Layer[Conv2d](in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        (activation): Layer[Sigmoid]()
        (normalization): Layer[BatchNorm2d](num_features=16)
      )
      (1): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[Sigmoid]()
        (normalization): Layer[BatchNorm2d](num_features=32)
      )
    )
    (postprocess): Layer[Identity]()
  )
  (bottleneck): ConvolutionalNeuralNetwork(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[Sigmoid]()
        (upsample): Layer[ConvTranspose2d](kernel_size=2, stride=2, padd

In [31]:
model[...].isinstance(torch.nn.BatchNorm2d).all \
    .initialize(dl.initializers.Constant(weight=1, bias=0))
    
print(model)

UNet2d(
  (encoder): ConvolutionalEncoder2d(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (layer): Layer[Conv2d](in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        (activation): Layer[Sigmoid]()
        (normalization): Layer[BatchNorm2d](num_features=16)
      )
      (1): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[Sigmoid]()
        (normalization): Layer[BatchNorm2d](num_features=32)
      )
    )
    (postprocess): Layer[Identity]()
  )
  (bottleneck): ConvolutionalNeuralNetwork(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
        (layer): Layer[Conv2d](in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        (activation): Layer[Sigmoid]()
        (upsample): Layer[ConvTranspose2d](kernel_size=2, stride=2, padd